# Exploring neural networks

## Problem 2

Given a dataset where each sample contains all 0s and randomly a number between 0 and 100 somewhere to replace the 0s.
Create a neural network that can distinguish between record no replaced number or a number below 50 and those that contain 0s and a number above 50.

## Problem 3

Given a datase where each sample contains random numbers between 0-10, create a neural network that can distinguish between record whose sum is greater than 20 and those that are less than 20.